<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/data-engineering/l1_demo_0_creating_a_table_with_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1 Demo 0: PostgreSQL and AutoCommits

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Postgresql_elephant.svg/1200px-Postgresql_elephant.svg.png" width="100" height="100">

In [1]:
#@title ## Setup database instance
#@markdown Executing this cell will setup PostgreSQL on the current Colab session.

#@markdown **Warning: This notebook is designed to be run in a Google Colab only**. *It installs packages on the system and requires sudo access. If you want to run it in a local Jupyter notebook, please proceed with caution.*
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS studentdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE studentdb;'

# Create student user
!sudo -u postgres psql -U postgres -c "CREATE USER student WITH ENCRYPTED PASSWORD 'student';"
!sudo -u postgres psql -U postgres -c "GRANT ALL PRIVILEGES ON DATABASE studentdb TO student;"

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

In [2]:
#@title ##Setup environment variables

#@markdown The following environmental variables are based on the PostgreSQL setup. You should edit to change the environemtn variables.

%env TFIO_DEMO_DATABASE_NAME=studentdb
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=student
%env TFIO_DEMO_DATABASE_PASS=student


env: TFIO_DEMO_DATABASE_NAME=studentdb
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=student
env: TFIO_DEMO_DATABASE_PASS=student


In [12]:
#@title ## Data loading queries
#@markdown - Drop tables
#@markdown - Create tables
#@markdown - Insert records
#@markdown - Queries

# DROP TABLES

songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS user_table_drop"
song_table_drop = "DROP TABLE IF EXISTS song_table_drop"
artist_table_drop = "DROP TABLE IF EXISTS artist_table_drop"
time_table_drop = "DROP TABLE IF EXISTS time_table_drop"

# CREATE TABLES
# CREATE FACT TABLE 
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS songplays(
                            songplay_id SERIAL PRIMARY KEY,
                            start_time timestamp,
                            user_id int NOT NULL,
                            level varchar,
                            artist_id varchar,
                            song_id varchar,
                            session_id int,
                            location text,
                            user_agent text)""")

# CREATE DIMENION TABLES
user_table_create = ("""CREATE TABLE IF NOT EXISTS users(
                        user_id int NOT NULL,
                        first_name varchar NOT NULL,
                        last_name varchar NOT NULL,
                        gender char,
                        level varchar,
                        PRIMARY KEY (user_id))""")

song_table_create = ("""CREATE TABLE IF NOT EXISTS songs(
                        song_id varchar NOT NULL,
                        title varchar,
                        artist_id varchar,
                        year int,
                        duration float,
                        PRIMARY KEY (song_id))""")

artist_table_create = ("""CREATE TABLE IF NOT EXISTS artists(
                        artist_id varchar NOT NULL,
                        name varchar,
                        location varchar,
                        lattitude numeric,
                        longitude numeric,
                        PRIMARY KEY (artist_id))""")

time_table_create = ("""CREATE TABLE IF NOT EXISTS time(
                        start_time timestamp NOT NULL,
                        hour int,
                        day int,
                        week int,
                        month int,
                        year int,
                        weekday varchar,
                        PRIMARY KEY (start_time))""")

# INSERT RECORDS

songplay_table_insert = ("""INSERT INTO songplays( start_time, user_id,level, artist_id, 
                            song_id, session_id, location, user_agent)
                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)""")

user_table_insert = ("""INSERT INTO users(user_id, first_name, last_name, gender,level)
                        VALUES(%s, %s, %s, %s, %s)
                        ON CONFLICT (user_id) 
                        DO UPDATE SET level = excluded.level""")

song_table_insert = ("""INSERT INTO songs(song_id, title, artist_id, year, duration)
                        VALUES(%s, %s, %s, %s, %s)
                        ON CONFLICT (song_id) 
                        DO NOTHING""")

artist_table_insert = ("""INSERT INTO  artists(artist_id, name,location,lattitude, longitude)
                          VALUES(%s, %s, %s, %s, %s)
                          ON CONFLICT (artist_id) 
                          DO NOTHING""")


time_table_insert = ("""INSERT INTO time(start_time,hour,day,week,month, year,weekday)
                        VALUES(%s, %s, %s, %s, %s, %s, %s)
                        ON CONFLICT (start_time) 
                        DO NOTHING""")

# FIND SONGS

song_select = ("""SELECT songs.song_id, artists.artist_id FROM songs 
                  JOIN artists ON  songs.artist_id=artists.artist_id
                  WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s;""")

# QUERY LISTS

create_table_queries = [songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]

print('Data loading queries are now available.')

Data loading queries are now available.


In [16]:
#@title ##Setup environment variables

#@markdown - Create `sparkifydb` database
#@markdown - Create test tables


import psycopg2

def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=postgres password=postgres")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS sparkifydb")
    cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
    
    return cur, conn


def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


cur, conn = create_database()
drop_tables(cur, conn)
create_tables(cur, conn)
conn.close()

print('Setup complete.')

Setup complete.


## Walk through the basics of PostgreSQL autocommits 

In [3]:
## import postgreSQL adapter for the Python
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Create a connection to the database
1. Connect to the local instance of PostgreSQL (*127.0.0.1*)
2. Use the database/schema from the instance. 
3. The connection reaches out to the database (*studentdb*) and use the correct privilages to connect to the database (*user and password = student*).

In [0]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")

### Use the connection to get a cursor that will be used to execute queries.

In [0]:
cur = conn.cursor()

### Create a database to work in

In [20]:
cur.execute("select * from test")

ProgrammingError: ignored

### Error occurs, but it was to be expected because table has not been created as yet. To fix the error, create the table. 

In [0]:
cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")

### Error indicates we cannot execute this query. Since we have not committed the transaction and had an error in the transaction block, we are blocked until we restart the connection.

In [0]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
cur = conn.cursor()

In our exercises instead of worrying about commiting each transaction or getting a strange error when we hit something unexpected, let's set autocommit to true. **This says after each call during the session commit that one action and do not hold open the transaction for any other actions. One action = one transaction.**

In this demo we will use automatic commit so each action is commited without having to call `conn.commit()` after each command. **The ability to rollback and commit transactions are a feature of Relational Databases.**

In [0]:
conn.set_session(autocommit=True)

In [0]:
cur.execute("select * from test")

In [0]:
cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")

### Once autocommit is set to true, we execute this code successfully. There were no issues with transaction blocks and we did not need to restart our connection. 

In [0]:
cur.execute("select * from test")

In [0]:
cur.execute("select count(*) from test")
print(cur.fetchall())